In [4]:
import torch
from math import log2
from torch import Tensor, sort

In [5]:
def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    # code here
    pass

In [6]:
ys_true = torch.rand(4)
ys_pred = torch.rand(4)
ys_true, ys_pred

(tensor([0.1914, 0.0989, 0.5567, 0.9545]),
 tensor([0.8793, 0.3513, 0.2643, 0.7537]))

In [8]:
y_sort, indices = torch.sort(ys_pred, descending=True)
y_sort, indices

(tensor([0.8793, 0.7537, 0.3513, 0.2643]), tensor([0, 3, 1, 2]))

In [9]:
ys_true[indices]

tensor([0.1914, 0.9545, 0.0989, 0.5567])

In [10]:
ys_pred, indices = torch.sort(ys_pred, descending=True)
ys_true = ys_true[indices]

num_correct_pairs = 0
len_preds = len(ys_pred)
for i in range(len_preds):
    if i==len_preds-1:
        break
    for j in range(i+1, len_preds):
        if ys_true[i] > ys_true[j]:
            num_correct_pairs += 1
ys_true, ys_pred

(tensor([0.1914, 0.9545, 0.0989, 0.5567]),
 tensor([0.8793, 0.7537, 0.3513, 0.2643]))

In [11]:
num_correct_pairs

3

In [12]:
def compute_gain(y_value: float, gain_scheme: str) -> float: # зачем exp
    return y_value if gain_scheme=='const' else 2**y_value - 1

In [13]:
compute_gain(2, 'const')

2

In [14]:
compute_gain(2, 'exp')

3

In [15]:
def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str) -> float:
    ys_pred, indices = torch.sort(ys_pred, descending=True)
    ys_true = ys_true[indices]

    sum_dcg = 0
    for i, y_true in enumerate(ys_true):
        gain = compute_gain(y_true, gain_scheme)
        sum_dcg += gain / log2(i + 2)
    return sum_dcg

In [16]:
ys_true = torch.FloatTensor([3.0, 1.0, 2.0, 1.0, 3.0])
ys_pred = torch.rand(5)
ys_true, ys_pred

(tensor([3., 1., 2., 1., 3.]),
 tensor([0.9367, 0.4576, 0.4359, 0.8286, 0.4015]))

In [17]:
preds_, indices = torch.sort(ys_pred, descending=True)
trues_ = ys_true[indices]
trues_, preds_

(tensor([3., 1., 1., 2., 3.]),
 tensor([0.9367, 0.8286, 0.4576, 0.4359, 0.4015]))

In [18]:
dcg(ys_true, ys_pred, 'const')

tensor(6.1528)

In [19]:
dcg(ys_true, ys_pred, 'exp')

tensor(12.1309)

In [20]:
def ndcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str = 'const') -> float:
    case_dcg = dcg(ys_true, ys_pred, gain_scheme)
    ideal_dcg = dcg(ys_true, ys_true, gain_scheme)
    return case_dcg / ideal_dcg

In [21]:
ys_true, ys_pred

(tensor([3., 1., 2., 1., 3.]),
 tensor([0.9367, 0.4576, 0.4359, 0.8286, 0.4015]))

In [22]:
trues_, preds_

(tensor([3., 1., 1., 2., 3.]),
 tensor([0.9367, 0.8286, 0.4576, 0.4359, 0.4015]))

In [23]:
ndcg(ys_true, ys_pred, 'const')

tensor(0.9169)

In [24]:
ndcg(ys_true, ys_pred, 'exp')

tensor(0.8833)

In [30]:
def precission_at_k(ys_true: Tensor, ys_pred: Tensor, k: int) -> float:
    infs = Tensor([float('-inf')] * (len(ys_true) - len(ys_pred)))
    ys_pred = torch.cat((ys_pred, infs))
    ys_pred, indices = torch.sort(ys_pred, descending=True)
    ys_true = ys_true[indices]
    
    pr_at_k = ys_true[:k].sum().item() / k
    return pr_at_k if pr_at_k != 0 else -1

In [31]:
ys_true = torch.randint(0, 2, (5,))
ys_pred = torch.rand(5)
ys_true, ys_pred

(tensor([1, 1, 0, 1, 1]), tensor([0.8868, 0.0735, 0.1343, 0.1117, 0.0245]))

In [32]:
preds_, indices = torch.sort(ys_pred, descending=True)
trues_ = ys_true[indices]
trues_, preds_

(tensor([1, 0, 1, 1, 1]), tensor([0.8868, 0.1343, 0.1117, 0.0735, 0.0245]))

In [33]:
k = 10

In [34]:
k = k if k < len(ys_true) else len(ys_true)

5

In [143]:
ext_ = Tensor([float('-inf')] * (len(trues_) - len(preds_)))

In [144]:
torch.cat((preds_, ext_))

torch.Size([5])

In [49]:
trues_[:k].sum().item() / k

0.4

In [50]:
preds_, indices = torch.sort(ys_pred, descending=True)
trues_ = ys_true[indices]
trues_, preds_

(tensor([0., 1., 0., 1.]), tensor([0.7651, 0.7647, 0.4005, 0.3113]))

In [51]:
precission_at_k(ys_true, ys_pred, 3)

0.3333333333333333

In [163]:
def reciprocal_rank(ys_true: Tensor, ys_pred: Tensor) -> float:
    ys_pred, indices = sort(ys_pred, descending=True)
    ys_true = ys_true[indices]
    ind_one = (ys_true == 1).nonzero(as_tuple=True)[0].item()
    return 1 / (ind_one + 1)

In [164]:
ys_true = torch.Tensor([0, 0, 1, 0, 0])
ys_pred = torch.rand(5)
ys_true, ys_pred

(tensor([0., 0., 1., 0., 0.]),
 tensor([0.8786, 0.8720, 0.5850, 0.6146, 0.1629]))

In [165]:
reciprocal_rank(ys_true, ys_pred)

0.25

In [193]:
def p_found(ys_true: Tensor, ys_pred: Tensor, p_break: float = 0.15 ) -> float:
    ys_pred, indices = sort(ys_pred, descending=True)
    ys_true = ys_true[indices]
    
    def calc_i_p_found(pred_p_look, pred_p_found, i, p_break):
        if pred_p_look == 0 or i > len(ys_true) - 1:
            return pred_p_found
        
        i_p_look = pred_p_look * (1 - ys_true[i-1].item()) * (1 - p_break)
        i_p_found = i_p_look * ys_true[i].item()
        return pred_p_found + calc_i_p_found(i_p_look, i_p_found, i + 1, p_break)
    
    i = 0
    zero_p_look = (1 - p_break)
    zero_p_found = zero_p_look * ys_true[0].item()
    return calc_i_p_found(zero_p_look, zero_p_found, i+1, p_break)
                            

In [208]:
ys_true = torch.rand(9)
ys_pred = torch.rand(9)
ys_true, ys_pred

(tensor([0.1997, 0.7797, 0.0721, 0.2334, 0.7874, 0.3536, 0.8629, 0.3887, 0.5590]),
 tensor([0.1834, 0.6663, 0.6942, 0.0840, 0.5720, 0.9291, 0.5139, 0.4721, 0.3459]))

In [209]:
preds_, indices = torch.sort(ys_pred, descending=True)
trues_ = ys_true[indices]
trues_, preds_

(tensor([0.3536, 0.0721, 0.7797, 0.7874, 0.8629, 0.3887, 0.5590, 0.1997, 0.2334]),
 tensor([0.9291, 0.6942, 0.6663, 0.5720, 0.5139, 0.4721, 0.3459, 0.1834, 0.0840]))

In [210]:
ys_true[0]

tensor(0.1997)

In [211]:
p_found(ys_true, ys_pred)

0.687580198514499


In [35]:
def average_precision(ys_true: Tensor, ys_pred: Tensor) -> float:
    # допишите ваш код здесь
    pass

In [44]:
ys_true = torch.Tensor([1, 1, 0, 0])
ys_pred = torch.rand(4)
ys_true, ys_pred

(tensor([1., 1., 0., 0.]), tensor([0.7647, 0.3113, 0.7651, 0.4005]))

In [45]:
preds_, indices = torch.sort(ys_pred, descending=True)
trues_ = ys_true[indices]
trues_, preds_

(tensor([0., 1., 0., 1.]), tensor([0.7651, 0.7647, 0.4005, 0.3113]))

In [48]:
ap = 0
for i in range(len(trues_)):
    print(trues_[:i+1])
    if trues_[i].item() == 1:
        ap += trues_[:i+1].sum().item() / (i + 1)
ap = ap / trues_.sum().item()

tensor([0.])
tensor([0., 1.])
tensor([0., 1., 0.])
tensor([0., 1., 0., 1.])


In [47]:
ap

0.5